In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import backend as bk
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from pandas import ExcelWriter
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

plt.style.use('ggplot')
pd.options.display.max_columns = None

scaler = MinMaxScaler()

## Import File and Reset Index

In [2]:
df = pd.read_excel('Competition1_raw_data.xlsx', na_values='-')
df.set_index('I1', inplace=True)

## Changing DataTypes

In [3]:
to_float = ['I3','P(IPO)','P(H)', 'P(L)', 'P(1Day)', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'T1', 'T2', 'T3', 'T4', 'T5', 'S1', 'S2', 'S3']

for col in to_float:
    df[col]=pd.to_numeric(df[col], errors='coerce')
df['C2'] = df['C2'].astype('bool') 

## Cleanining/Imputing

For our predictor variables only

In [4]:
df = df.dropna(subset=['P(1Day)'])
## Creating Necessary Variables Y1, Y2, C3', C5', and C6'
## Code found on https://stackoverflow.com/questions/19913659/pandas-conditional-creation-of-a-series-dataframe-column
df['P(Mid)'] = (df['P(H)'] + df['P(L)']) / 2
df['Y1'] = np.where(df['P(IPO)'] < df['P(Mid)'], 1,0)
df['Y2'] = np.where(df['P(IPO)'] < df['P(1Day)'], 1,0)
df["C3_Prime"] = np.where(df['C3'] > 0, 1, 0)
df["C5_Prime"] = df['C5'] / df['C6']
df["C6_Prime"] = np.where(df['P(IPO)'] > df['P(Mid)'], ((df['P(IPO)'] - df['P(Mid)']) / df['P(Mid)']) * 100,0)

## Now we need to change our industry columns

In [5]:
df['industry'] = df['I3'].apply(bk.assign_sector)

df['industry'] = df['industry'].replace(['Mining', 'Wholesale Trade', 'Finance/Insurance/Realestate', 'Non Classafiable Establishments',
                                            'Agriculture/Forestry/Fishing', 'Construction', 'NaN'],'Other')
df['industry'].value_counts()

df['industry'] = df['industry'].replace({'Manufacturing': 1, 'Services': 2, 'Transportation/Communications/Utulities': 3, 'Retail Trade': 4,'Other': 5})

df['industry'] = df['industry'].fillna(5)

df['industry_1'] = np.where(df['industry'] ==1, 1, 0)
df['industry_2'] = np.where(df['industry'] ==2, 1, 0)
df['industry_3'] = np.where(df['industry'] ==3, 1, 0)
df['industry_4'] = np.where(df['industry'] ==4, 1, 0)
df['industry_5'] = np.where(df['industry'] ==5, 1, 0)


## Now we need to create our ratio columns


In [6]:
t2_mean = df['T2'].mean()
df['T2'] = np.where(df['T2'] ==0, t2_mean, df['T2'])
df["S1_Prime"]= df['S1']/df['T2']
df["S2_Prime"]= df['S2']/df['T2']
df["S3_Prime"]= df['S3']/df['T2']
df["T3_Prime"]= df['T3']/df['T2']
df["T4_Prime"]= df['T4']/df['T1']
df["T5_Prime"]= df['T5']/df['T2']

## No longer need these columns

In [7]:
df = df.drop(columns = ['I2', 'I3'])

## Getting rid of our 0 value columns

In [8]:
#T3 need to get rid of thier zeros.
t3_mean = df['T3'].mean()
df['T3'] = np.where(df['T3'] ==0, t3_mean, df['T3'])
print(t3_mean)

#S1 need to get rid of thier zeros.
s1_mean = df['S1'].mean()
df['S1'] = np.where(df['S1'] ==-1, s1_mean, df['S1'])
print(s1_mean)

#T5 need to get rid of thier zeros.
t5_mean = df['T5'].mean()
df['T5'] = np.where(df['T5'] ==-1, s1_mean, df['T5'])
print(t5_mean)

11463.39393939394
68.67575757575757
669.8909090909091


## More Imputation

In [9]:
#nromalize qnd impute C7 values
df['C7'] = np.log(df['C7'])
c7_mean = df['C7'].mean()
print(c7_mean)
df['C7'] = df['C7'].fillna(c7_mean)

#normalize and impute C3 values
df['C3'] = np.cbrt(df['C3'])
c3_mean = df['C3'].mean()
print(c3_mean)
df['C3'] = df['C3'].fillna(c3_mean)

4.592588484196865
-0.21843318437028425


## Creyting More Variables for us to use


## Normalize the Rest of the DF

In [10]:
df['market_cap'] =df['P(1Day)'] * df['C6']
df['market_cap'] = np.where(df['market_cap'] ==0, 0.001, df['market_cap'])
df.describe()

,P(IPO),P(H),P(L),P(1Day),C1,C3,C4,C5,C6,C7,T1,T2,T3,T4,T5,S1,S2,S3,P(Mid),Y1,Y2,C3_Prime,C5_Prime,C6_Prime,industry,industry_1,industry_2,industry_3,industry_4,industry_5,S1_Prime,S2_Prime,S3_Prime,T3_Prime,T4_Prime,T5_Prime,market_cap
count,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,6.600000e+02,6.600000e+02,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,6.600000e+02
mean,14.034621,15.610394,13.635394,25.934766,149.728788,-0.218433,0.007282,4.996285e+07,1.258248e+07,4.592588,468.701515,12888.641524,11515.500275,295.957576,669.996478,68.781327,120.896970,145.516667,14.622894,0.500000,0.698485,0.477273,4.417255,5.404645,2.048485,0.448485,0.303030,0.087879,0.072727,0.087879,0.005308,0.009055,0.011516,0.891504,0.626720,0.052171,3.050895e+08
std,5.954214,6.613816,5.790112,73.234948,152.817467,1.534849,0.033318,1.055377e+08,2.539940e+07,1.916640,176.374430,5415.681452,4805.168278,122.172959,298.043412,39.316670,85.542267,69.824545,6.187137,0.500379,0.459264,0.499862,5.133537,10.877569,1.272118,0.497716,0.459917,0.283333,0.259885,0.283333,0.001755,0.002990,0.002842,0.071672,0.077686,0.007347,9.190430e+08
min,5.000000,0.000000,5.000000,0.000000,10.000000,-9.229642,-0.162352,3.693227e+06,8.000000e+05,-2.603690,132.000000,3271.000000,3003.000000,0.000000,68.675758,9.000000,20.000000,40.000000,4.000000,0.000000,0.000000,0.000000,0.283223,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000093,0.002152,0.005326,0.000000,0.000000,-0.000089,1.000000e-03
25%,10.000000,13.000000,11.000000,11.000000,85.000000,-0.926368,-0.013927,1.877568e+07,5.124546e+06,3.724681,356.000000,9275.500000,8297.000000,214.000000,463.750000,45.000000,74.000000,100.000000,12.000000,0.000000,0.000000,0.000000,2.850622,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.004131,0.006966,0.009579,0.883417,0.580329,0.047544,6.566256e+07
50%,13.750000,15.500000,13.000000,14.845000,107.000000,-0.218433,0.009125,2.745438e+07,7.500000e+06,4.592588,446.000000,12141.500000,10853.000000,280.000000,627.000000,60.000000,101.500000,135.000000,14.250000,0.500000,1.000000,0.000000,3.731035,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005027,0.008577,0.011301,0.897294,0.628861,0.051927,1.229588e+08
75%,17.000000,17.000000,15.000000,20.485000,155.250000,0.738329,0.031571,5.003986e+07,1.200000e+07,5.706326,553.250000,15275.000000,13793.500000,355.250000,801.250000,85.000000,144.000000,174.000000,16.000000,1.000000,1.000000,1.000000,4.857904,7.692308,2.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.006254,0.010521,0.012848,0.909693,0.674461,0.056489,2.401875e+08
max,85.000000,135.000000,108.000000,1159.200562,2087.000000,15.692704,0.092896,2.138085e+09,4.212336e+08,10.331464,1750.000000,49056.000000,43952.000000,1058.000000,2729.000000,309.000000,944.000000,883.000000,121.500000,1.000000,1.000000,1.000000,99.787255,100.000000,5.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.012839,0.024605,0.029397,1.490540,0.826347,0.088329,1.610452e+10


## Standradize the data

In [11]:
pos_skew = ['P(IPO)', 'P(L)', 'C1', 'C5', 'C6', 'T1', 'T2', 'T3', 'T5', 'S1', 'S2', 'S3', 'P(Mid)', 'C5_Prime', 
            'S2_Prime','S3_Prime', 'S1_Prime', 'market_cap' ]

for col in pos_skew:
    df[col]=np.log(df[col])

cbrt_skew = ['P(1Day)', 'T4', 'C6_Prime', 'P(H)']

for col in cbrt_skew:
    df[col]=np.cbrt(df[col])

over_log = ['C5', 'C6']
for col in over_log:
    df[col]=1/np.log(df[col])

df['T4'] = np.sqrt(df['T4'])

neg_skew = ['T3_Prime', 'T4_Prime']

for col in neg_skew:
    df[col]=df[col]**2
    

df['industry_1'] = df['industry_1'].astype('category')
df['industry_2'] = df['industry_2'].astype('category')
df['industry_3'] = df['industry_3'].astype('category')
df['industry_4'] = df['industry_4'].astype('category')
df['industry_5'] = df['industry_5'].astype('category')
df.describe()

/opt/tljh/user/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in log
  """


,P(IPO),P(H),P(L),P(1Day),C1,C3,C4,C5,C6,C7,T1,T2,T3,T4,T5,S1,S2,S3,P(Mid),Y1,Y2,C3_Prime,C5_Prime,C6_Prime,industry,S1_Prime,S2_Prime,S3_Prime,T3_Prime,T4_Prime,T5_Prime,market_cap
count,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,659.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000
mean,2.570174,2.468196,2.556056,2.595374,4.757714,-0.218433,0.007282,0.351260,0.361369,4.592588,6.085846,9.384331,9.272666,2.547272,6.420257,4.095896,4.629703,4.892455,2.629442,0.500000,0.698485,0.477273,1.321676,0.872089,2.048485,-5.288790,-4.754628,-4.491876,0.799909,0.398804,0.052171,18.567660
std,0.374586,0.276076,0.324272,0.869526,0.670366,1.534849,0.033318,0.005743,0.006058,1.916640,0.359248,0.401475,0.398817,0.219957,0.418741,0.522101,0.555689,0.410897,0.312893,0.500379,0.459264,0.499862,0.508141,1.158446,1.272118,0.324152,0.315494,0.233634,0.099443,0.090072,0.007347,2.275603
min,1.609438,0.000000,1.609438,0.000000,2.302585,-9.229642,-0.162352,0.326023,0.334600,-2.603690,4.882802,8.092851,8.007367,0.000000,4.229396,2.197225,2.995732,3.688879,1.386294,0.000000,0.000000,0.000000,-1.261521,0.000000,1.000000,-6.436265,-6.141157,-5.235159,0.000000,0.000000,-0.000089,-6.907755
25%,2.302585,2.351335,2.397895,2.223980,4.442651,-0.926368,-0.013927,0.347806,0.358270,3.724681,5.874931,9.135132,9.023647,2.445695,6.139345,3.806662,4.304065,4.605170,2.484907,0.000000,0.000000,0.000000,1.047537,0.000000,1.000000,-5.488490,-4.966767,-4.648159,0.780425,0.336782,0.047544,18.000029
50%,2.620874,2.493315,2.564949,2.457688,4.672829,-0.218433,0.009125,0.352024,0.362065,4.592588,6.100319,9.404384,9.292197,2.557759,6.440947,4.094345,4.620047,4.905275,2.656603,0.500000,1.000000,0.000000,1.316685,0.000000,2.000000,-5.291951,-4.758642,-4.482895,0.805136,0.395466,0.051927,18.627359
75%,2.833213,2.571282,2.708050,2.736184,5.045033,0.738329,0.031571,0.354826,0.365286,5.706326,6.315810,9.633973,9.531953,2.661270,6.686169,4.442651,4.969813,5.159055,2.772589,1.000000,1.000000,1.000000,1.580607,1.974023,2.000000,-5.074419,-4.554401,-4.354551,0.827541,0.454897,0.056489,19.296930
max,4.442651,5.129928,4.682131,10.504761,7.643483,15.692704,0.092896,0.368168,0.383214,10.331464,7.467371,10.800718,10.690853,3.192133,7.911691,5.733341,6.850126,6.783325,4.799914,1.000000,1.000000,1.000000,4.603040,4.641589,5.000000,-4.355265,-3.704811,-3.526857,2.221710,0.682850,0.088329,23.502366


In [12]:
cols = ['C1','C2', 'C3', 'C4', 'C5', 'C6','C7',	'T1',	'T2',	'T3',	'T4',	'T5', 
            'S1',	'S2',	'S3',	'C3_Prime',	'C5_Prime',	'C6_Prime', 'S2_Prime', 
            'S3_Prime',	'T3_Prime',	'T4_Prime',	'T5_Prime', 'market_cap']

for col in cols:
    df[[col]] = scaler.fit_transform(df[[col]])

df.describe()

,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,C4,C5,C6,C7,T1,T2,T3,T4,T5,S1,S2,S3,P(Mid),Y1,Y2,C3_Prime,C5_Prime,C6_Prime,industry,S1_Prime,S2_Prime,S3_Prime,T3_Prime,T4_Prime,T5_Prime,market_cap
count,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000,659.000000,660.000000,660.000000,660.000000,660.000000,660.000000,660.000000
mean,2.570174,2.468196,2.556056,2.595374,0.459685,0.859091,0.361571,0.664586,0.598806,0.550651,0.556335,0.465472,0.476936,0.471513,0.797984,0.594972,0.536937,0.423924,0.388947,2.629442,0.500000,0.698485,0.477273,0.440476,0.187886,2.048485,-5.288790,0.569102,0.435101,0.360042,0.584029,0.591054,0.837728
std,0.374586,0.276076,0.324272,0.869526,0.125516,0.348192,0.061585,0.130531,0.136263,0.124622,0.148173,0.138997,0.148262,0.148619,0.068906,0.113717,0.147648,0.144170,0.132785,0.312893,0.500379,0.459264,0.499862,0.086646,0.249580,1.272118,0.324152,0.129495,0.136764,0.044759,0.131907,0.083097,0.074830
min,1.609438,0.000000,1.609438,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.386294,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,-6.436265,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.302585,2.351335,2.397895,2.223980,0.400694,1.000000,0.333166,0.581494,0.516867,0.486897,0.489238,0.383866,0.384908,0.378716,0.766163,0.518684,0.455143,0.339439,0.296108,2.484907,0.000000,0.000000,0.000000,0.393731,0.000000,1.000000,-5.488490,0.482029,0.343616,0.351272,0.493201,0.538725,0.819062
50%,2.620874,2.493315,2.564949,2.457688,0.443791,1.000000,0.361571,0.671804,0.616940,0.564966,0.556335,0.471072,0.484342,0.478791,0.801270,0.600590,0.536498,0.421419,0.393090,2.656603,0.500000,1.000000,0.000000,0.439625,0.000000,2.000000,-5.291951,0.567454,0.440358,0.362395,0.579141,0.588295,0.839691
75%,2.833213,2.571282,2.708050,2.736184,0.513481,1.000000,0.399961,0.759743,0.683427,0.631220,0.642437,0.554447,0.569128,0.568136,0.833697,0.667185,0.634998,0.512164,0.475102,2.772589,1.000000,1.000000,1.000000,0.484627,0.425290,2.000000,-5.074419,0.651285,0.515487,0.372479,0.666175,0.639898,0.861709
max,4.442651,5.129928,4.682131,10.504761,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,4.799914,1.000000,1.000000,1.000000,1.000000,1.000000,5.000000,-4.355265,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [13]:
cols_test_y1 = ['C1','C2', 'C3', 'C4', 'C5', 'C6','C7',	'T1',	'T2',	'T3',	'T4',	'T5', 
            'S1',	'S2',	'S3',	'C3_Prime',	'C5_Prime', 'S2_Prime', 
            'S3_Prime',	'T3_Prime',	'T4_Prime',	'T5_Prime',	'industry_1',	'industry_2',	'industry_3',	'industry_4',	'industry_5', 'market_cap']

cols_test_y2 = ['C1','C2', 'C3', 'C4', 'C5', 'C6','C7',	'T1',	'T2',	'T3',	'T4',	'T5', 
            'S1',	'S2',	'S3',	'C3_Prime',	'C5_Prime',	'C6_Prime', 'S2_Prime', 
            'S3_Prime',	'T3_Prime',	'T4_Prime',	'T5_Prime',	'industry_1',	'industry_2',	'industry_3',	'industry_4',	'industry_5']

dataset_y1 = df[cols_test_y1].copy()
dataset_y2 = df[cols_test_y2].copy()
target_y1 = df[['Y1']].copy()
target_y2 = df[['Y2']].copy()

#s1Prime causes us to have an error for some reason

## Y1 Columns

In [14]:
dataset_y1 = dataset_y1.values
target_y1 = target_y1.values

# create a base classifier used to evaluate a subset of attributes
model = LogisticRegression()
# create the RFE model and select 3 attributes
rfe = RFE(model, 6)
rfe = rfe.fit(dataset_y1, target_y1)
# summarize the selection of the attributes
print(rfe.support_)
print(rfe.ranking_)

[False False  True False False False False False False False False False
 False False False False  True  True False False  True  True False False
 False False False  True]
[13 10  1  2  3 21 11  8 17 18  5 15  4  7 22 14  1  1 16  9  1  1 19 23
 12  6 20  1]


/opt/tljh/user/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## Y2

In [15]:
dataset_y2 = dataset_y2 = dataset_y2.values
target_y2 = target_y2.values

# create a base classifier used to evaluate a subset of attributes
model = LogisticRegression()
# create the RFE model and select 3 attributes
rfe = RFE(model, 6)
rfe = rfe.fit(dataset_y2, target_y2)
# summarize the selection of the attributes
print(rfe.support_)
print(rfe.ranking_)

[False False False  True False False False False False False False False
  True False False False  True  True  True False False False False False
 False  True False False]
[16  7  3  1 12 11 22  8 21 14  4 15  1  5 20 18  1  1  1 13  6 17 19 10
  9  1 23  2]


/opt/tljh/user/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## Collumns we want to keep

In [18]:
df_output = df.copy()
cols_keep = ['C3', 'C5_Prime', 'S2_Prime', 'T4_Prime','T5_Prime', 'market_cap', 'C4','S1', 'C5_Prime','C6_Prime', 'S2_Prime', 'industry_2', 'Y1', 'Y2']
df_output = df.copy()
df_output = df_output[cols_keep]
df_output.head()

,C3,C5_Prime,S2_Prime,T4_Prime,T5_Prime,market_cap,C4,S1,C5_Prime,C6_Prime,S2_Prime,industry_2,Y1,Y2
I1,,,,,,,,,,,,,,
AATI,0.430848,0.445611,0.596171,0.600637,0.614565,0.840448,0.749962,0.545771,0.445611,0.480750,0.596171,0,0,1
ABPI,0.323211,0.639234,0.673466,0.608782,0.582281,0.775391,0.583748,0.584103,0.639234,0.000000,0.673466,0,1,0
ACAD,0.327229,0.422227,0.438935,0.593885,0.699549,0.796932,0.717213,0.541173,0.422227,0.000000,0.438935,0,1,0
ACHN,0.331453,0.418676,0.492391,0.426455,0.698035,0.813684,0.714502,0.617853,0.418676,0.000000,0.492391,0,1,1
ACLI,0.386873,0.439407,0.632380,0.505314,0.550107,0.883571,0.499347,0.567704,0.439407,0.368403,0.632380,0,0,1


## Write to Datafile

In [19]:
df_output.to_csv('datasets/group11.csv', sep=',')